In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

from pathlib import Path

from sklearn.metrics import log_loss

path = Path('/kaggle/input/tabular-playground-series-nov-2022/')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
submission = pd.read_csv(path / 'sample_submission.csv', index_col='id')
labels = pd.read_csv(path / 'train_labels.csv', index_col='id')

# the ids of the submission rows (useful later)
sub_ids = submission.index

# the ids of the labeled rows (useful later)
gt_ids = labels.index 

# list of files in the submission folder
subs = sorted(os.listdir(path / 'submission_files'))

Please, upvote my notebook if You like it!:)

Let's try to find result as a weighted sum of our submissions. We have sorted list of submissions. So, we can take the weighted sum of several best solutions. As coefficients, for example, take 1/logloss. 

In [ ]:
subs1 = pd.Series(subs).apply(lambda x: float(x[:-4]))
subs2 = subs1.apply(lambda x: 1/x)
# subs3 = subs2.apply(lambda x: x**2)

In [ ]:
def weighted_sum(lst):
    s = np.zeros(lst[0].shape)
    
    for i in range(len(lst)):
        s += lst[i] * subs2[i]
    return s / subs2[:len(lst)].sum()

def blend(n):
    s = []
    for i in range(n):
        sub = pd.read_csv(path / 'submission_files' / subs[i], index_col='id')
        s.append(sub)

    return weighted_sum(s)


score = log_loss(labels, blend(3).loc[gt_ids])

print(f'blend score: {score:.10f}')

Let's make a plot to see what is the best number of terms to use

In [ ]:
blend_plot = [log_loss(labels, blend(i).loc[gt_ids]) for i in range(1,20)]

In [ ]:
pd.Series(blend_plot).plot()

We will use 3 first submissions

In [ ]:
preds = blend(3).loc[sub_ids]
preds

In [ ]:
preds.to_csv('submission.csv')